In [1]:
import sys
sys.path.insert(0, '/mnt/nas/users/mjm/LDPUts')

from discretizer import discretizer
from client import client
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt
from server import server_ell2, server_multinomial_genRR, server_multinomial_bitflip
from data_generator import data_generator
from discretizer import discretizer
import time
import numpy as np
from scipy.stats import chi2
from utils import chi_sq_dist

In [10]:
n_test = 100
n_permutation = 99
significance_level = 0.05
alphabet_size = 100
sample_size = 50000

###################
###################

data_gen = data_generator(device)

p1 = torch.ones(alphabet_size).div(alphabet_size)

bump_size = 0.005
p2 = p1.add(
    torch.remainder(
    torch.tensor(range(alphabet_size)),
    2
    ).add(-1/2).mul(2).mul(bump_size)
)
print(p2)



tensor([0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050,
        0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150,
        0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050,
        0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150,
        0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050,
        0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150,
        0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050,
        0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150,
        0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050,
        0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150,
        0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050, 0.0150, 0.0050,
        0.0150])


## privacy level = 1.0

In [9]:
p_value_array_priv_10_n_20000 = np.zeros([n_test, 6])

privacy_level = 2.0
LDPclient = client(device, privacy_level)

server_elltwo = server_ell2(device, privacy_level)
server_elltwo_disc = server_ell2(device, privacy_level)

server_genrr = server_multinomial_genRR(device, privacy_level)
server_bitflip = server_multinomial_bitflip(device, privacy_level)



t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    data_y = data_gen.generate_multinomial_data(p1, sample_size)
    data_z = data_gen.generate_multinomial_data(p2, sample_size)
    LDPclient.load_data_multinomial(data_y, data_z, alphabet_size)
    
    data_lapu_y, data_lapu_z = LDPclient.release_LapU()
    server_elltwo.load_private_data_multinomial(data_lapu_y, data_lapu_z, alphabet_size)
    p_value_array_priv_10_n_20000[i,0] = server_elltwo.release_p_value_permutation(n_permutation)
    
    data_disclapu_y, data_disclapu_z = LDPclient.release_DiscLapU()
    server_elltwo_disc.load_private_data_multinomial(data_disclapu_y, data_disclapu_z, alphabet_size)
    p_value_array_priv_10_n_20000[i,1] = server_elltwo_disc.release_p_value_permutation(n_permutation)
    
    data_genrr_y, data_genrr_z = LDPclient.release_genRR()
    server_genrr.load_private_data_multinomial(data_genrr_y, data_genrr_z, alphabet_size)
    p_value_array_priv_10_n_20000[i,2] = server_genrr.release_p_value_permutation(n_permutation)
    p_value_array_priv_10_n_20000[i,3] = server_genrr.release_p_value()
    
    data_bitflip_y, data_bitflip_z = LDPclient.release_bitFlip()
    server_bitflip.load_private_data_multinomial(data_bitflip_y, data_bitflip_z, alphabet_size)
    p_value_array_priv_10_n_20000[i,4] = server_bitflip.release_p_value_permutation(n_permutation)
    p_value_array_priv_10_n_20000[i,5] = server_bitflip.release_p_value()
elapsed = time.time() - t

1th test
2th test
3th test
4th test
5th test
6th test
7th test
8th test
9th test
10th test
11th test
12th test
13th test
14th test
15th test
16th test
17th test
18th test
19th test
20th test
21th test
22th test
23th test
24th test
25th test
26th test
27th test
28th test
29th test
30th test
31th test
32th test
33th test
34th test
35th test
36th test
37th test
38th test
39th test
40th test
41th test
42th test
43th test
44th test
45th test
46th test
47th test
48th test
49th test
50th test


In [24]:
def geom(alpha):

    root_1 = torch.tensor(alpha).exp().add(
            torch.tensor(2*alpha).exp().sub(4).sqrt()
    ).div(2)
    
    root_2 = torch.tensor(alpha).exp().sub(
            torch.tensor(2*alpha).exp().sub(4).sqrt()
    ).div(2)
    return (1-min(root_1, root_2))

def geom_var(alpha):
    geom_param = geom(alpha)
    return(geom_param.mul(-1).add(1).div(geom_param.square()))

In [28]:
geom_var(1)

tensor(1.3922)

In [33]:
geom_param.mul(-1).add(1).div(geom_param.square())

tensor(1.3922)

In [15]:
geom_param.mul(-1).add(1).div(geom_param)

tensor(0.7815)

In [16]:
geometric_generator = torch.distributions.geometric.Geometric(geom(1))

In [20]:
geometric_generator.sample([100]).mean()

tensor(0.8700)

In [5]:
(p_value_array_priv_10_n_20000<significance_level).mean(axis=0)

array([0.15, 0.19, 0.23, 0.22, 0.34, 0.36])

In [7]:
data_y.float().var()

tensor(837.2700, device='cuda:0')

In [9]:
data_geomu_y[1,:]

tensor([-1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997,  8.4003, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997,  8.4003,  8.4003, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997, -1.5997,  8.4003, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997, -1.5997,  8.4003, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        -1.5997, -1.5997, -1.5997,  8.4003, -1.5997, -1.5997, 18.4003,  8.4003,
        -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,  8.4003, -1.5997,
        -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,
        18.4003, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997, -1.5997,  8.4003,
         8.4003, -1.5997, -1.5997, -1.59

In [12]:
n_test = 100
n_permutation = 99
significance_level = 0.05
alphabet_size = 100
sample_size = 50000

p_value_array_priv_10_geom = np.zeros([n_test, 6])
privacy_level = 2.0
LDPclient = client(device, privacy_level)

server_elltwo = server_ell2(device, privacy_level)
server_elltwo_disc = server_ell2(device, privacy_level)

server_genrr = server_multinomial_genRR(device, privacy_level)
server_bitflip = server_multinomial_bitflip(device, privacy_level)

t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    data_y = data_gen.generate_multinomial_data(p1, sample_size)
    data_z = data_gen.generate_multinomial_data(p2, sample_size)
    LDPclient.load_data_multinomial(data_y, data_z, alphabet_size)
    
    data_geomu_y, data_geomu_z = LDPclient.release_geomU()
    server_elltwo.load_private_data_multinomial(data_geomu_y, data_geomu_z, alphabet_size)
    p_value_array_priv_10_geom[i,0] = server_elltwo.release_p_value_permutation(n_permutation)
elapsed = time.time() - t

1th test
2th test
3th test
4th test
5th test
6th test
7th test
8th test
9th test
10th test
11th test
12th test
13th test
14th test
15th test
16th test
17th test
18th test
19th test
20th test
21th test
22th test
23th test
24th test
25th test
26th test
27th test
28th test
29th test
30th test
31th test
32th test
33th test
34th test
35th test
36th test
37th test
38th test
39th test
40th test
41th test
42th test
43th test
44th test
45th test
46th test
47th test
48th test
49th test
50th test
51th test
52th test
53th test
54th test
55th test
56th test
57th test
58th test
59th test
60th test
61th test
62th test
63th test
64th test
65th test
66th test
67th test
68th test
69th test
70th test
71th test
72th test
73th test
74th test
75th test
76th test
77th test
78th test
79th test
80th test
81th test
82th test
83th test
84th test
85th test
86th test
87th test
88th test
89th test
90th test
91th test
92th test
93th test
94th test
95th test
96th test
97th test
98th test
99th test
100th test


In [14]:
(p_value_array_priv_10_geom<0.05).mean(axis=0)

array([1., 1., 1., 1., 1., 1.])

In [35]:
n_test = 100
n_permutation = 99
significance_level = 0.05
sample_size = 7000

p_value_array_priv_10_geom_n_10000 = np.zeros([n_test, 6])
privacy_level = 2.0
LDPclient = client(device, privacy_level)

server_elltwo = server_ell2(device, privacy_level)
server_elltwo_disc = server_ell2(device, privacy_level)

server_genrr = server_multinomial_genRR(device, privacy_level)
server_bitflip = server_multinomial_bitflip(device, privacy_level)

t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    data_y = data_gen.generate_multinomial_data(p1, sample_size)
    data_z = data_gen.generate_multinomial_data(p2, sample_size)
    LDPclient.load_data_multinomial(data_y, data_z, alphabet_size)
    
    data_geomu_y, data_geomu_z = LDPclient.release_geomU()
    server_elltwo.load_private_data_multinomial(data_geomu_y, data_geomu_z, alphabet_size)
    p_value_array_priv_10_geom_n_10000[i,0] = server_elltwo.release_p_value_permutation(n_permutation)
elapsed = time.time() - t

1th test
2th test
3th test
4th test
5th test
6th test
7th test
8th test
9th test
10th test
11th test
12th test
13th test
14th test
15th test
16th test
17th test
18th test
19th test
20th test
21th test
22th test
23th test
24th test
25th test
26th test
27th test
28th test
29th test
30th test
31th test
32th test
33th test
34th test
35th test
36th test
37th test
38th test
39th test
40th test
41th test
42th test
43th test
44th test
45th test
46th test
47th test
48th test
49th test
50th test
51th test
52th test
53th test
54th test
55th test
56th test
57th test
58th test
59th test
60th test
61th test
62th test
63th test
64th test
65th test
66th test
67th test
68th test
69th test
70th test
71th test
72th test
73th test
74th test
75th test
76th test
77th test
78th test
79th test
80th test
81th test
82th test
83th test
84th test
85th test
86th test
87th test
88th test
89th test
90th test
91th test
92th test
93th test
94th test
95th test
96th test
97th test
98th test
99th test
100th test


In [36]:
(p_value_array_priv_10_geom_n_10000<0.05).mean(axis=0)

array([0.79, 1.  , 1.  , 1.  , 1.  , 1.  ])

In [33]:
data_geomu_y.mean()

tensor(0.0937, device='cuda:0')